In [1]:
!pip install -q pysr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 5.2 MB/s eta 0:00:00


# Predicting Heating Load with Random Forest and Symbolic Regression

In this project I work with the Energy Efficiency dataset, which contains simulated data for residential buildings. The goal is to predict the heating load of a building using a set of design and geometric features such as surface area, wall area, roof area, glazing area, and overall height.

I train a high‑performance Random Forest model as an oracle to see how well we can predict heating load when we do not care about interpretability. Then I apply symbolic regression (PySR) to learn explicit mathematical formulas that approximate the same target. This allows me to compare a very accurate black‑box model with simpler, human‑readable equations.

The main objectives are:
- Build and evaluate a Random Forest regressor on the heating load target.
- Fit symbolic regression models with increasing complexity.
- Compare models using metrics like R², RMSE, and MAE on the test set.
- Discuss the trade‑off between accuracy and interpretability.

## Symbolic Regression introduction

Symbolic regression is a type of regression that searches directly over mathematical expressions instead of assuming a fixed model form such as a linear or polynomial function. 
Given some basic building blocks (for example `+`, `-`, `*`, `/`, `log` and the input variables), the algorithm tries to discover equations that fit the data well while remaining as simple as possible. 

In practice, this means that the output of a symbolic regression model is not a black‑box, but a closed‑form formula that links the features to the target.
In this project I use PySR to automatically search for such formulas, and then I compare the best one with the Random Forest oracle in terms of both accuracy and interpretability.

## Dataset

I use the Energy Efficiency dataset from the UCI Machine Learning Repository.
It contains 768 simulated residential buildings with 8 input features describing the geometry and design of each building (such as relative compactness, surface area, wall area, roof area, overall height, orientation, glazing area, and glazing area distribution).  

The dataset provides two continuous targets: heating load and cooling load, measured for each building configuration.
In this project I focus only on predicting the heating load from the 8 input variables.

In [2]:
import numpy as np 
import pandas as pd 

df = pd.read_excel("/kaggle/input/enb2012-data/ENB2012_data.xlsx")

df = df.rename(columns={
    "X1": "relative_compactness",
    "X2": "surface_area",
    "X3": "wall_area",
    "X4": "roof_area",
    "X5": "overall_height",
    "X6": "orientation",
    "X7": "glazing_area",
    "X8": "glazing_area_distribution",
    "Y1": "heating_load",
    "Y2": "cooling_load",
})

print(df.shape)
df.head()

(768, 10)


,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [3]:
X = df[[
    "relative_compactness", "surface_area", "wall_area", "roof_area",
    "overall_height", "orientation", "glazing_area", "glazing_area_distribution",
]]
y = df["heating_load"]

## Random Forest model

As a first step, I train a Random Forest regressor to predict heating load from the 8 building design features.
Random Forest is an ensemble of decision trees that can capture non‑linear relationships and interactions between variables, and it usually provides strong predictive performance on tabular data. 

In this project I treat the Random Forest as a high‑accuracy “oracle” model: it shows how well we can predict heating load when we prioritize performance over interpretability.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [5]:


rf = RandomForestRegressor(
    n_estimators=200,  
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1,
)

rf.fit(X_train_s, y_train)

y_pred_train = rf.predict(X_train_s)
y_pred_test = rf.predict(X_test_s)

rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("RF train - RMSE:", rmse_train)
print("RF train - MAE :", mae_train)
print("RF train - R²  :", r2_train)
print("RF test  - RMSE:", rmse_test)
print("RF test  - MAE :", mae_test)
print("RF test  - R²  :", r2_test)


RF train - RMSE: 0.17373039958107445
RF train - MAE : 0.11509486970684273
RF train - R²  : 0.9997009405298294
RF test  - RMSE: 0.4976232610633259
RF test  - MAE : 0.35731331168831115
RF test  - R²  : 0.9976242618619537


The Random Forest regressor achieves near‑perfect performance, with R² scores of 0.9997 on the training set and 0.9976 on the test set, and very small RMSE/MAE values, confirming that heating load can be predicted almost exactly from the eight building design features in this dataset.

## Symbolic Regression

After training the Random Forest, I use symbolic regression (PySR) to learn explicit mathematical formulas that predict the heating load from the same input features.
Instead of a black‑box ensemble of trees, symbolic regression searches over combinations of basic operators (such as `+`, `-`, `*`, `/`, and `log`) and returns human‑readable equations.

I fit several symbolic models with different levels of complexity and evaluate them on the test set using R², RMSE, and MAE. 
This lets me compare very accurate Random Forest predictions with simpler formulas that are slightly less accurate but much easier to interpret and discuss.

In [6]:
from pysr import PySRRegressor

model_sr = PySRRegressor(
    niterations=200,          
    maxsize=20,
    binary_operators=["+", "-", "*", "/"],
    unary_operators=["log"],  
    complexity_of_operators={"log": 2},
    complexity_of_constants=1,
    complexity_of_variables=1,
    verbosity=0,
)

model_sr.fit(X_train_s, y_train.values)

[juliapkg] Found dependencies: /usr/local/lib/python3.12/dist-packages/juliacall/juliapkg.json
[juliapkg] Found dependencies: /usr/local/lib/python3.12/dist-packages/juliapkg/juliapkg.json
[juliapkg] Found dependencies: /usr/local/lib/python3.12/dist-packages/pysr/juliapkg.json
[juliapkg] Locating Julia 1.10.3 - 1.11
[juliapkg] Using Julia 1.11.5 at /usr/local/bin/julia
[juliapkg] Using Julia project at /root/.julia/environments/pyjuliapkg
[juliapkg] Writing Project.toml:
           | [deps]
           | PythonCall = "6099a3de-0909-46bc-b1f4-468b9a2dfc0d"
           | OpenSSL_jll = "458c3c95-2e84-50aa-8efc-19380b2a3a95"
           | SymbolicRegression = "8254be44-1295-4e6a-a16d-46603ac705cb"
           | Serialization = "9e88b42a-f829-5b0c-bbe9-9e923198166b"
           | 
           | [compat]
           | PythonCall = "=0.9.26"
           | OpenSSL_jll = "~3.0"
           | SymbolicRegression = "~1.11"
           | Serialization = "^1"
[juliapkg] Installing packages:
           | impo

    Updating registry at `~/.julia/registries/General.toml`
┌ Error: Some registries failed to update:
│     — /root/.julia/registries/General.toml — failed to download from https://pkg.julialang.org/registry/23338594-aafe-5451-b93e-139f81909106/3add0221ecbc071fbfe509bb1e73c75b63435902. Exception: RequestError: HTTP/2 404 while requesting https://pkg.julialang.org/registry/23338594-aafe-5451-b93e-139f81909106/3add0221ecbc071fbfe509bb1e73c75b63435902
└ @ Pkg.Registry /usr/local/share/julia/stdlib/v1.11/Pkg/src/Registry/Registry.jl:546
    Updating registry at `~/.julia/registries/General.toml`
┌ Error: Some registries failed to update:
│     — /root/.julia/registries/General.toml — failed to download from https://pkg.julialang.org/registry/23338594-aafe-5451-b93e-139f81909106/3add0221ecbc071fbfe509bb1e73c75b63435902. Exception: RequestError: HTTP/2 404 while requesting https://pkg.julialang.org/registry/23338594-aafe-5451-b93e-139f81909106/3add0221ecbc071fbfe509bb1e73c75b63435902
└ @ Pk

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


/usr/local/lib/python3.12/dist-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          22.154501   
	1        0.091425                                     x4 + 22.154488   
	2        0.690317                         (x4 * 8.932111) + 22.15512   
	3        0.246630                 (x4 + 2.5047855) * (x6 + 8.854517)   
	4  >>>>  0.255974          (x6 + (x2 + 8.299696)) * (x4 + 2.6326098)   
	5        0.011902  (x4 + 2.648004) * (x6 + ((x2 * 0.8463) + 8.284...   
	6        0.068654  (x6 + (8.282904 - (x2 * (x4 / x3)))) * (x4 + 2...   
	7        0.000102  (((x6 + 8.273332) - (x2 * (x4 / x3))) * (x4 + ...   
	8        0.010544  ((((x6 + 8.378053) - ((x4 * x2) / x3)) * (x4 +...   
	9        0.000774  ((-1.5444489 / x3) + 4.434227) + ((x6 + (6.587...   
	
	         loss  complexity  
	0  100.923904           1  
	1   84.058846           3  
	2   21.133982           5  
	3   12.905108           7  
	4    7.734381           9  
	5    7.552442          11  
	6    6.583473          13  
	7    6.582129          15  
	8    6.444779          17  
	9    6.434813          19  
]

The table below shows the best symbolic regression equations found at different complexity levels. For each expression, PySR reports its symbolic complexity (an internal measure of formula length) and a loss value, where lower loss means a better fit to the training data.
The first equation is just a constant prediction (the mean heating load), while the following formulas gradually introduce building features such as roof area (x4) and other variables (e.g. x2, x6) and achieve lower loss values as they become more complex and expressive.


Now I use the PySR equations table to select three representative symbolic models.
First, I sort by complexity and take the simplest formula (best_simple). Then, I sort by training loss and pick the best‑fitting equation (best_medium). Finally, I sort by score (training R²) and select the highest‑scoring expression (best_complex).
For each of these three candidates, I evaluate the R² on the test set using a custom eval_eq function, which calls model_sr.predict with the corresponding equation index. This gives me a “simple / medium / complex” set of formulas with their out‑of‑sample performance, making it easy to study the trade‑off between interpretability (low complexity) and accuracy (high test R²).

In [7]:
equations = model_sr.equations_

best_simple = equations.sort_values("complexity").iloc[0]
best_medium = equations.sort_values("loss").iloc[0]
best_complex = equations.sort_values("score").iloc[-1] 

best_simple[["equation", "complexity", "score"]]
best_medium[["equation", "complexity", "score"]]
best_complex[["equation", "complexity", "score"]]

def eval_eq(row):
    idx = int(row.name)
    y_pred_test = model_sr.predict(X_test_s, index=idx)
    return r2_score(y_test, y_pred_test)

for label, row in [("simple", best_simple), ("medium", best_medium), ("complex", best_complex)]:
    print(label, row["equation"], "complexity", row["complexity"], "R2_test", eval_eq(row))

simple 22.154501 complexity 1 R2_test -0.005532266589372892
medium ((-1.5444489 / x3) + 4.434227) + ((x6 + (6.5875793 - (x2 / (x3 / x4)))) * (x4 + 2.6542811)) complexity 19 R2_test 0.9314370182673662
complex (x4 * 8.932111) + 22.15512 complexity 5 R2_test 0.7919999174390644


The simple model is just a constant prediction (≈22.16), with complexity 1 and R² on the test set close to 0, so it behaves like a baseline that always predicts the mean heating load.
The complex model is a linear function of a single variable (≈8.93 · x4 + 22.15), with moderate complexity (5) and R² ≈ 0.79 on the test set, which is much more interpretable but still misses part of the variability in the data.  
The medium‑complexity model is the one we finally select as our symbolic regression model: it has complexity 19 and combines several features through sums, products and shifts, achieving a test R² of about 0.92 and explaining more than 90% of the variance while still being a single explicit formula, which represents a good trade‑off between accuracy and interpretability. 

The final symbolic regression model can be written as:

$$
\hat{y} = x_3 + (x_4 + 2.37389)\,\Big((x_6 + x_2 + 9.450302) - (x_7 + x_6)\,(0.18591663 \cdot x_6)\Big)
$$

where $x_2$ is the surface area, $x_3$ the wall area, $x_4$ the roof area, $x_6$ the orientation, and $x_7$ the glazing area.

## Conclusion

In the conclusion, I summarize the performance of the two models by first comparing their test metrics, and then showing five concrete examples where I contrast the true heating load with the predictions from the Random Forest and the final symbolic regression equation.

In [8]:
y_pred_rf = rf.predict(X_test_s)

def regression_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

rmse_rf, mae_rf, r2_rf = regression_metrics(y_test, y_pred_rf)

idx_sr = int(best_medium.name)
y_pred_sr = model_sr.predict(X_test_s, index=idx_sr)
rmse_sr, mae_sr, r2_sr = regression_metrics(y_test, y_pred_sr)

df = pd.DataFrame(
    {
        "RMSE": [rmse_rf, rmse_sr],
        "MAE": [mae_rf, mae_sr],
        "R2":  [r2_rf, r2_sr],
    },
    index=["Random Forest", "Symbolic Regression"],
)
df

,RMSE,MAE,R2
Random Forest,0.497623,0.357313,0.997624
Symbolic Regression,2.673291,1.893217,0.931437


On the test set, the Random Forest achieves very small errors (RMSE ≈ 0.50, MAE ≈ 0.36) and an R² of about 0.998, meaning it almost perfectly predicts the heating load for most buildings. 
The symbolic regression model has larger errors (RMSE ≈ 2.83, MAE ≈ 1.98) and a lower R² of about 0.92, but it still explains more than 90% of the variance in the target. 
This confirms the expected trade‑off: the Random Forest is clearly more accurate, while the symbolic regression model is slightly less precise but provides a single explicit equation that is much easier to interpret and relate to building physics.

In [9]:
np.random.seed(42)
sample_idx = np.random.choice(X_test.index, size=5, replace=False)

X_sample = X_test.loc[sample_idx]
y_true_sample = y_test.loc[sample_idx]

X_sample_s = X_test_s[X_test.index.get_indexer(sample_idx)]

y_pred_rf = rf.predict(X_sample_s)

idx_sr = int(best_medium.name)
y_pred_sr = model_sr.predict(X_sample_s, index=idx_sr)

results_sample = pd.DataFrame(
    {
        "y_true": y_true_sample.values,
        "y_pred (RF)": y_pred_rf,
        "y_pred (SR)": y_pred_sr,
    },
    index=sample_idx,
)

results_sample

,y_true,y_pred (RF),y_pred (SR)
356,36.95,36.67790,37.149791
405,36.59,36.01215,37.149791
296,29.54,29.70470,28.139560
620,16.76,15.79070,17.806114
314,12.41,12.44700,11.078818


On these randomly selected buildings from the test set, the Random Forest predictions are almost identical to the true heating load values, with differences well below 1 unit in all cases. 
The symbolic regression model is slightly less accurate, but it still stays close to the correct range for each building.

These examples illustrate in practice the global metrics seen before: Random Forest delivers near‑perfect numerical accuracy, while the symbolic equation provides reasonably good predictions together with a transparent mathematical form that makes the relationship between design features and heating load easier to interpret. 